# Set-Up

In [6]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model
import pmdarima as pm
%matplotlib inline
from datetime import datetime
import functools
import warnings
warnings.filterwarnings("ignore")

In [8]:
stocks = pd.read_csv('ohlc_data.csv')
stocks = stocks[['Date', 'Adj Close', 'ticker']]
stocks = stocks.pivot('Date','ticker', 'Adj Close')
stocks

ticker,AAPL,AMGN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-10-02,0.371812,51.270111,41.145496,38.098213,9.826061,40.923904,20.023323,30.562424,89.962357,34.223339,21.206047,72.623405,24.768309,27.057945,25.594147,16.025986,17.687233,26.910892,32.208397,18.697594,3.987808,19.345022,29.491835,11.066429,16.422474,25.862204,31.325428
2000-10-03,0.342106,52.049896,40.165859,39.035389,10.059161,41.476929,20.066637,30.417343,88.792709,35.037216,21.905556,68.154297,24.884058,26.858318,25.190031,15.832685,17.797550,27.589617,32.208397,17.887224,4.137349,19.558187,28.829100,11.001145,16.509134,25.862204,31.029131
2000-10-04,0.362229,50.246658,39.809448,39.850319,10.077092,43.182095,19.590235,30.852579,85.332619,36.176609,22.310522,70.504402,25.925718,26.223154,24.719379,16.096281,17.650467,28.029890,32.017963,17.531464,4.056346,19.931231,28.387276,10.713874,17.137436,26.297596,31.240788
2000-10-05,0.338272,51.952415,39.891190,39.809566,10.041229,42.444721,19.749039,31.916456,89.572479,34.996513,21.574207,69.772423,25.308435,27.021646,24.109013,16.623438,17.724001,27.369492,32.779640,17.511694,4.286890,20.748386,28.497723,10.948911,18.123524,27.298988,31.198456
2000-10-06,0.340188,49.174477,37.888451,39.768806,9.897784,41.430851,19.489172,31.481234,84.894005,33.246677,21.500582,71.506126,24.652576,26.803879,23.261293,16.430153,17.576923,27.057646,33.106079,17.570993,3.956653,21.014845,27.577246,10.753041,17.599468,26.384684,30.817465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,130.347565,218.065536,115.889725,219.690002,175.734024,43.406982,82.363388,173.550003,254.586807,266.731110,207.354233,119.538940,45.708645,149.175735,122.741341,52.248150,209.566040,170.009567,74.801056,219.583740,140.954880,133.819366,136.430771,335.167328,57.073406,38.774467,141.545731
2020-12-24,131.352829,218.055725,116.286095,217.149994,176.856873,43.563477,82.017365,173.729996,254.298904,267.828461,206.900009,120.301132,46.199398,149.696091,122.201576,52.602509,208.943314,170.527435,75.157425,221.302490,140.795792,135.173843,136.964844,338.344391,56.957249,38.531334,141.822449
2020-12-28,136.050766,218.759995,117.286942,216.089996,175.684784,43.935146,81.604057,178.860001,257.703979,266.177521,207.304871,120.426559,46.199398,150.402985,123.006310,53.311226,211.542877,170.713089,75.448151,223.498138,141.621078,136.116104,137.657120,343.467377,57.092766,38.570240,143.522339


# Returns

In [9]:
return_data = np.log(stocks).diff() # rescale to faciliate optimization
return_data = return_data.dropna()
return_data

ticker,AAPL,AMGN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-10-03,-0.083268,0.015095,-0.024097,0.024301,0.023446,0.013423,0.002161,-0.004758,-0.013087,0.023503,0.032454,-0.063513,0.004662,-0.007405,-0.015915,-0.012135,0.006218,0.024908,0.000000,-0.044308,0.036814,0.010959,-0.022728,-0.005917,0.005263,0.000000,-0.009504
2000-10-04,0.057157,-0.035259,-0.008913,0.020662,0.001781,0.040289,-0.024027,0.014207,-0.039748,0.032002,0.018318,0.033901,0.041008,-0.023933,-0.018861,0.016512,-0.008299,0.015832,-0.005930,-0.020090,-0.019773,0.018894,-0.015444,-0.026460,0.037351,0.016695,0.006798
2000-10-05,-0.068427,0.033384,0.002051,-0.001023,-0.003565,-0.017223,0.008074,0.033901,0.048491,-0.033164,-0.033560,-0.010436,-0.024098,0.029996,-0.025002,0.032225,0.004157,-0.023842,0.023510,-0.001128,0.055279,0.040181,0.003883,0.021700,0.055945,0.037372,-0.001356
2000-10-06,0.005650,-0.054953,-0.051509,-0.001024,-0.014389,-0.024177,-0.013246,-0.013730,-0.053645,-0.051294,-0.003419,0.024544,-0.026256,-0.008092,-0.035795,-0.011695,-0.008333,-0.011459,0.009909,0.003381,-0.080163,0.012761,-0.032833,-0.018051,-0.029342,-0.034066,-0.012287
2000-10-09,-0.019914,0.001980,-0.022914,-0.025948,0.001810,-0.045514,0.019077,0.006125,-0.033862,-0.003679,0.011915,0.016565,-0.022153,-0.010892,-0.019132,-0.010753,0.006256,-0.004757,-0.009909,-0.025058,-0.017475,-0.014475,-0.021593,0.000607,-0.015000,0.003295,-0.008276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,-0.007000,0.008785,0.020735,0.004151,0.010366,-0.006961,0.015643,0.018024,0.025113,-0.000852,0.009810,0.002343,0.008626,-0.005120,0.027561,0.006047,0.000472,-0.004530,0.004272,-0.013125,-0.004856,-0.001539,0.010713,0.007672,-0.000509,0.015163,-0.006819
2020-12-24,0.007683,-0.000045,0.003414,-0.011629,0.006369,0.003599,-0.004210,0.001037,-0.001132,0.004106,-0.002193,0.006356,0.010679,0.003482,-0.004407,0.006759,-0.002976,0.003041,0.004753,0.007797,-0.001129,0.010071,0.003907,0.009434,-0.002037,-0.006290,0.001953
2020-12-28,0.035141,0.003225,0.008570,-0.004893,-0.006649,0.008495,-0.005052,0.029101,0.013301,-0.006183,0.001955,0.001042,0.000000,0.004711,0.006564,0.013383,0.012365,0.001088,0.003861,0.009873,0.005844,0.006947,0.005042,0.015028,0.002376,0.001009,0.011915


# Volatility

In [10]:
vol = return_data.rolling(252).std()
vol

ticker,AAPL,AMGN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-10-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-10-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-10-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-10-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-10-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,0.029378,0.023965,0.039722,0.055363,0.028528,0.026489,0.038994,0.030647,0.032924,0.028090,0.028534,0.025946,0.033882,0.019028,0.034214,0.021975,0.025248,0.023633,0.020056,0.027756,0.026169,0.020425,0.031365,0.030468,0.015403,0.030571,0.019652
2020-12-24,0.029360,0.023965,0.039721,0.055365,0.028529,0.026490,0.038994,0.030647,0.032923,0.028091,0.028535,0.025949,0.033886,0.019029,0.034208,0.021976,0.025248,0.023634,0.020059,0.027756,0.026168,0.020434,0.031365,0.030473,0.015404,0.030565,0.019652
2020-12-28,0.029432,0.023966,0.039725,0.055365,0.028532,0.026495,0.038994,0.030699,0.032932,0.028093,0.028534,0.025948,0.033884,0.019031,0.034211,0.021989,0.025258,0.023633,0.020060,0.027761,0.026166,0.020434,0.031366,0.030486,0.015402,0.030565,0.019664


# Rolling Modelling

In [11]:
def rolling_modelling(returns, n, ticker, p=3, q=5):
    train = returns[ticker][:-n]
    model = arch_model(train, p = p, q = q, vol='GARCH')
    model_fit = model.fit(disp = 'off')
    pred = model_fit.forecast(horizon = n)
    rolling_predictions=np.sqrt(pred.variance.values[-1,:])
    rolling_predictions = pd.Series(rolling_predictions, index=returns.index[-n:], name=ticker)
    return rolling_predictions

In [60]:
def rolling_modelling_all(returns, pred_start, pred_end, p, q):
    sub=return_data.loc[pred_start:pred_end]
    len_pred=len(sub)
    list_out = [rolling_modelling(returns, len_pred, ticker, p=p, q=q) for ticker in returns.columns]
    compile_out=functools.reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True), list_out)
    
    df_vol = compile_out # predicted vol
    df_vol_trans=pd.DataFrame(df_vol.stack().rename_axis(index=['Date', 'ticker']))
    for ticker in df_vol.columns:
        df_vol_trans.loc[:, ticker] = df_vol_trans.iloc[:, 0]
    df_vol_trans = df_vol_trans.iloc[: , 1:] # predicted vol (transposed per row from test)
    
    corr=returns.rolling(31).corr() # rolling 30 day correlation
    pred_cov=corr*df_vol*df_vol_trans
    return pred_cov[pred_start:pred_end]

In [61]:
def rolling_modelling_all_test(returns, pred_start, pred_end, p, q):
    sub=return_data.loc[pred_start:pred_end]
    len_pred=len(sub)
    list_out = [rolling_modelling(returns, len_pred, ticker, p=p, q=q) for ticker in returns.columns]
    compile_out=functools.reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True), list_out)
    
    df_vol = compile_out # predicted vol
    return df_vol

In [62]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import itertools

def run_rolling_modelling_all_test(return_data, pred_start, pred_end, p, q, actual, root):
    pred = rolling_modelling_all_test(return_data, pred_start=pred_start, pred_end=pred_end, p=p, q=q)
    actual = actual
    df = pd.DataFrame(index=["p="+str(p)+", q="+str(q)])
    for ticker in actual.columns:
        df[ticker]=mean_squared_error(pred[[ticker]], actual[[ticker]], squared=root)
    return df

def run_rolling_modelling_all_test_mae(return_data, pred_start, pred_end, p, q, actual):
    pred = rolling_modelling_all_test(return_data, pred_start=pred_start, pred_end=pred_end, p=p, q=q)
    actual = actual
    df = pd.DataFrame(index=["p="+str(p)+", q="+str(q)])
    for ticker in actual.columns:
        df[ticker]=mean_absolute_error(pred[[ticker]], actual[[ticker]])
    return df

def compile_scores(return_data, pred_start, pred_end, actual, root):
    ps = [1, 2, 3, 4, 5]
    qs = [1, 2, 3, 4, 5]
    combis = list(itertools.product(ps, qs))
    list_df = [run_rolling_modelling_all_test(return_data, pred_start, pred_end, *i, actual, root) for i in combis]
    concat = pd.concat(list_df)
    return concat

def compile_scores_mae(return_data, pred_start, pred_end, actual):
    ps = [1, 2, 3, 4, 5]
    qs = [1, 2, 3, 4, 5]
    combis = list(itertools.product(ps, qs))
    list_df = [run_rolling_modelling_all_test_mae(return_data, pred_start, pred_end, *i, actual) for i in combis]
    concat = pd.concat(list_df)
    return concat
    
scores_MSE = compile_scores(return_data["2001-01-01":"2020-12-31"], 
                        pred_start='2016-01-01', pred_end='2020-12-31', 
                        actual=vol["2016-01-01":"2020-12-31"],
                        root =False)

scores_RMSE = compile_scores(return_data["2001-01-01":"2020-12-31"], 
                        pred_start='2016-01-01', pred_end='2020-12-31', 
                        actual=vol["2016-01-01":"2020-12-31"],
                        root= True)

scores_MAE = compile_scores_mae(return_data["2001-01-01":"2020-12-31"], 
                        pred_start='2016-01-01', pred_end='2020-12-31', 
                        actual=vol["2016-01-01":"2020-12-31"])

/usr/local/lib/python3.7/dist-packages/arch/univariate/base.py:755: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/arch/univariate/base.py:755: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/arch/univariate/base.py:755: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/arch/univariate/base.py:755: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning,
/usr/local/lib/python

# MSE, RMSE & MAE Scores

In [63]:
scores_MSE

,AAPL,AMGN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
"p=1, q=1",0.008654,0.004787,0.010871,0.013552,0.004620,0.009991,0.008789,0.007618,0.008087,0.007577,0.008655,0.004728,0.007837,0.003719,0.011791,0.004929,0.005671,0.005192,0.005155,0.005212,0.004052,0.016669,0.008469,0.007821,0.003853,0.005253,0.002961
"p=1, q=2",0.008561,0.004773,0.010781,0.013543,0.004390,0.009898,0.008781,0.007588,0.008036,0.007516,0.008621,0.004744,0.007737,0.058546,0.011708,0.004793,0.005644,0.005190,0.005147,0.005658,0.004043,0.004202,0.008411,0.007697,0.003787,0.005259,0.071854
"p=1, q=3",0.008479,0.004759,0.010703,0.013531,0.004578,0.009747,0.008781,0.007548,0.007974,0.007453,0.008575,0.004702,0.007653,0.013653,0.011620,0.004775,0.005616,0.005200,0.005147,0.005634,0.004033,0.004192,0.008346,0.007737,0.003719,0.005259,0.002960
"p=1, q=4",0.008390,0.004748,0.010619,0.013518,0.004554,0.006070,0.008785,0.007500,0.007906,0.007363,0.008521,0.004683,0.007551,0.855867,0.011525,0.004756,0.005588,0.005291,0.005244,0.005607,0.004000,0.004185,0.008281,0.007696,0.003649,0.005258,0.002952
"p=1, q=5",0.008299,0.004996,0.010530,0.013503,0.004529,0.009512,0.008796,0.007461,0.007837,0.007941,0.008463,0.004664,0.007447,0.003666,0.011425,0.004734,0.005562,0.005186,0.005140,0.005365,0.003986,0.004177,0.008216,0.007655,0.004419,0.005257,0.002950
"p=2, q=1",0.008650,0.004786,0.010834,0.013551,0.004619,0.009981,0.008788,0.007615,0.008078,0.007568,0.008650,0.004727,0.007826,0.003718,0.011764,0.004804,0.005669,0.005193,0.005153,0.005664,0.004051,0.004207,0.008461,0.007815,0.003843,0.005260,0.163374
"p=2, q=2",0.008552,0.004771,0.010760,0.013541,0.004596,0.009889,0.008781,0.007584,0.008027,0.007507,0.008615,0.004716,0.007738,0.061429,0.011681,0.004790,0.005641,0.005190,0.005149,0.005656,0.004042,0.004202,0.008403,0.007773,0.003778,0.005258,0.002970
"p=2, q=3",0.008468,0.004757,0.010680,0.013647,0.004575,0.010058,0.009188,0.007542,0.007930,0.007435,0.008547,0.004727,0.007631,0.003691,0.011592,0.004773,0.005612,0.005200,0.005147,0.005631,0.004032,0.004249,0.008630,0.007730,0.003695,0.005259,0.002960
"p=2, q=4",0.008379,0.004832,0.010595,0.013519,0.004456,0.009615,0.008788,0.007485,0.006293,0.007758,0.008509,0.004737,0.007538,0.003677,0.011496,0.038254,0.005584,0.005185,0.005153,0.005599,0.003999,0.004184,0.008262,0.007689,0.003630,0.005258,0.002952
"p=2, q=5",0.008286,0.004735,0.010505,0.013506,0.004514,0.009509,0.008799,0.008812,0.007818,0.007272,0.008464,0.004736,0.007433,0.003662,0.011394,0.004737,0.005558,0.005187,0.004975,0.005576,0.003984,0.004176,0.008197,0.007648,0.003564,0.005257,0.002950


In [64]:
scores_RMSE

,AAPL,AMGN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
"p=1, q=1",0.000075,0.000023,0.000118,0.000184,0.000021,0.000100,0.000077,0.000058,0.000065,0.000057,0.000075,0.000022,0.000061,0.000014,0.000139,0.000024,0.000032,0.000027,0.000027,0.000027,0.000016,0.000278,0.000072,0.000061,0.000015,0.000028,0.000009
"p=1, q=2",0.000073,0.000023,0.000116,0.000183,0.000019,0.000098,0.000077,0.000058,0.000065,0.000056,0.000074,0.000023,0.000060,0.003428,0.000137,0.000023,0.000032,0.000027,0.000026,0.000032,0.000016,0.000018,0.000071,0.000059,0.000014,0.000028,0.005163
"p=1, q=3",0.000072,0.000023,0.000115,0.000183,0.000021,0.000095,0.000077,0.000057,0.000064,0.000056,0.000074,0.000022,0.000059,0.000186,0.000135,0.000023,0.000032,0.000027,0.000026,0.000032,0.000016,0.000018,0.000070,0.000060,0.000014,0.000028,0.000009
"p=1, q=4",0.000070,0.000023,0.000113,0.000183,0.000021,0.000037,0.000077,0.000056,0.000063,0.000054,0.000073,0.000022,0.000057,0.732509,0.000133,0.000023,0.000031,0.000028,0.000027,0.000031,0.000016,0.000018,0.000069,0.000059,0.000013,0.000028,0.000009
"p=1, q=5",0.000069,0.000025,0.000111,0.000182,0.000021,0.000090,0.000077,0.000056,0.000061,0.000063,0.000072,0.000022,0.000055,0.000013,0.000131,0.000022,0.000031,0.000027,0.000026,0.000029,0.000016,0.000017,0.000068,0.000059,0.000020,0.000028,0.000009
"p=2, q=1",0.000075,0.000023,0.000117,0.000184,0.000021,0.000100,0.000077,0.000058,0.000065,0.000057,0.000075,0.000022,0.000061,0.000014,0.000138,0.000023,0.000032,0.000027,0.000027,0.000032,0.000016,0.000018,0.000072,0.000061,0.000015,0.000028,0.026691
"p=2, q=2",0.000073,0.000023,0.000116,0.000183,0.000021,0.000098,0.000077,0.000058,0.000064,0.000056,0.000074,0.000022,0.000060,0.003774,0.000136,0.000023,0.000032,0.000027,0.000027,0.000032,0.000016,0.000018,0.000071,0.000060,0.000014,0.000028,0.000009
"p=2, q=3",0.000072,0.000023,0.000114,0.000186,0.000021,0.000101,0.000084,0.000057,0.000063,0.000055,0.000073,0.000022,0.000058,0.000014,0.000134,0.000023,0.000031,0.000027,0.000026,0.000032,0.000016,0.000018,0.000074,0.000060,0.000014,0.000028,0.000009
"p=2, q=4",0.000070,0.000023,0.000112,0.000183,0.000020,0.000092,0.000077,0.000056,0.000040,0.000060,0.000072,0.000022,0.000057,0.000014,0.000132,0.001463,0.000031,0.000027,0.000027,0.000031,0.000016,0.000018,0.000068,0.000059,0.000013,0.000028,0.000009
"p=2, q=5",0.000069,0.000022,0.000110,0.000182,0.000020,0.000090,0.000077,0.000078,0.000061,0.000053,0.000072,0.000022,0.000055,0.000013,0.000130,0.000022,0.000031,0.000027,0.000025,0.000031,0.000016,0.000017,0.000067,0.000058,0.000013,0.000028,0.000009


In [65]:
scores_MAE

,AAPL,AMGN,AXP,BA,CAT,CSCO,CVX,DIS,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
"p=1, q=1",0.007734,0.004405,0.010466,0.007915,0.003903,0.009085,0.006170,0.007159,0.007868,0.007281,0.008374,0.003592,0.007013,0.002821,0.011202,0.004406,0.004841,0.004253,0.004892,0.004072,0.003427,0.008485,0.008226,0.007316,0.003439,0.003676,0.002451
"p=1, q=2",0.007589,0.004373,0.010288,0.007861,0.003617,0.008972,0.006131,0.007112,0.007796,0.007191,0.008327,0.003646,0.006878,0.028194,0.011082,0.004139,0.004789,0.004246,0.004883,0.004818,0.003409,0.003087,0.008146,0.007151,0.003376,0.003656,0.034172
"p=1, q=3",0.007484,0.004344,0.010155,0.007796,0.003827,0.008769,0.006109,0.007056,0.007713,0.007109,0.008266,0.003540,0.006778,0.008342,0.010961,0.004111,0.004736,0.004264,0.004882,0.004777,0.003394,0.003056,0.008059,0.007198,0.003311,0.003656,0.002470
"p=1, q=4",0.007382,0.004321,0.010021,0.007730,0.003791,0.005514,0.006106,0.006994,0.007623,0.006968,0.008197,0.003516,0.006664,0.811103,0.010835,0.004080,0.004687,0.004385,0.004961,0.004738,0.003332,0.003042,0.007965,0.007160,0.003237,0.003653,0.002443
"p=1, q=5",0.007287,0.004530,0.009888,0.007667,0.003755,0.008520,0.006110,0.006933,0.007522,0.007549,0.008127,0.003489,0.006554,0.002735,0.010707,0.004047,0.004646,0.004240,0.004872,0.004388,0.003317,0.003030,0.007884,0.007125,0.004064,0.003649,0.002418
"p=2, q=1",0.007727,0.004404,0.010381,0.007908,0.003901,0.009073,0.006165,0.007152,0.007855,0.007266,0.008366,0.003589,0.006996,0.002819,0.011156,0.004158,0.004838,0.004254,0.004889,0.004819,0.003424,0.003098,0.008214,0.007304,0.003429,0.003658,0.065057
"p=2, q=2",0.007575,0.004371,0.010250,0.007853,0.003858,0.008961,0.006127,0.007105,0.007784,0.007176,0.008319,0.003561,0.006879,0.029993,0.011039,0.004134,0.004784,0.004246,0.004884,0.004815,0.003407,0.003085,0.008135,0.007238,0.003366,0.003657,0.002494
"p=2, q=3",0.007470,0.004341,0.010117,0.007708,0.003823,0.009007,0.007879,0.007047,0.007637,0.007086,0.008217,0.003621,0.006750,0.002773,0.010920,0.004106,0.004729,0.004265,0.004881,0.004774,0.003392,0.003380,0.008378,0.007187,0.003279,0.003657,0.002472
"p=2, q=4",0.007369,0.004391,0.009984,0.007736,0.003687,0.008625,0.006106,0.006954,0.005555,0.007377,0.008174,0.003644,0.006649,0.002751,0.010795,0.023724,0.004681,0.004236,0.004884,0.004727,0.003333,0.003040,0.007940,0.007150,0.003218,0.003654,0.002444
"p=2, q=5",0.007273,0.004298,0.009851,0.007678,0.003736,0.008513,0.006113,0.008328,0.007497,0.006856,0.008123,0.003637,0.006538,0.002729,0.010666,0.004050,0.004640,0.004241,0.004731,0.004698,0.003313,0.003026,0.007859,0.007116,0.003157,0.003650,0.002419


In [102]:
scores_MSE["MEAN"]=scores_MSE.mean(axis=1)
score_table_MSE = pd.concat([scores_MSE.idxmin(), scores_MSE.min(axis=0)], axis=1)
score_table_MSE.columns = ["model", "MSE"]
score_table_MSE = score_table_MSE.reindex(np.roll(score_table_MSE.index, shift=1))
score_table_MSE

,model,MSE
MEAN,"p=3, q=5",0.006669
AAPL,"p=5, q=5",0.008246
AMGN,"p=2, q=5",0.004735
AXP,"p=5, q=5",0.010429
BA,"p=3, q=5",0.013427
CAT,"p=1, q=2",0.004390
CSCO,"p=1, q=4",0.006070
CVX,"p=4, q=4",0.008780
DIS,"p=5, q=5",0.007405
GS,"p=2, q=4",0.006293


In [100]:
scores_RMSE["MEAN"]=scores_RMSE.mean(axis=1)
score_table_RMSE = pd.concat([scores_RMSE.idxmin(), scores_RMSE.min(axis=0)], axis=1)
score_table_RMSE.columns = ["model", "RMSE"]
score_table_RMSE = score_table_RMSE.reindex(np.roll(score_table_RMSE.index, shift=1))
score_table_RMSE

,model,RMSE
MEAN,"p=3, q=5",0.000051
AAPL,"p=5, q=5",0.000068
AMGN,"p=2, q=5",0.000022
AXP,"p=5, q=5",0.000109
BA,"p=3, q=5",0.000180
CAT,"p=1, q=2",0.000019
CSCO,"p=1, q=4",0.000037
CVX,"p=4, q=4",0.000077
DIS,"p=5, q=5",0.000055
GS,"p=2, q=4",0.000040


In [101]:
scores_MAE["MEAN"]=scores_MAE.mean(axis=1)
score_table_MAE = pd.concat([scores_MAE.idxmin(), scores_MAE.min(axis=0)], axis=1)
score_table_MAE.columns = ["model", "MAE"]
score_table_MAE = score_table_MAE.reindex(np.roll(score_table_MAE.index, shift=1))
score_table_MAE

,model,MAE
MEAN,"p=3, q=5",0.005700
AAPL,"p=5, q=5",0.007227
AMGN,"p=2, q=5",0.004298
AXP,"p=5, q=5",0.009739
BA,"p=5, q=5",0.007641
CAT,"p=1, q=2",0.003617
CSCO,"p=1, q=4",0.005514
CVX,"p=1, q=4",0.006106
DIS,"p=5, q=5",0.006866
GS,"p=2, q=4",0.005555


# Covariance Matrix

In [70]:
start="2001-01-01"
pred_cov_2016=rolling_modelling_all(return_data[start:"2016-12-31"], pred_start='2016-01-01', pred_end='2016-12-31', p=3, q=5)
pred_cov_2017=rolling_modelling_all(return_data[start:"2017-12-31"], pred_start='2017-01-01', pred_end='2017-12-31', p=3, q=5)
pred_cov_2018=rolling_modelling_all(return_data[start:"2018-12-31"], pred_start='2018-01-01', pred_end='2018-12-31', p=3, q=5)
pred_cov_2019=rolling_modelling_all(return_data[start:"2019-12-31"], pred_start='2019-01-01', pred_end='2019-12-31', p=3, q=5)
pred_cov_2020=rolling_modelling_all(return_data[start:"2020-12-31"], pred_start='2020-01-01', pred_end='2020-12-31', p=3, q=5)

In [71]:
final_cov_pred=pd.concat([pred_cov_2016,pred_cov_2017,pred_cov_2018,pred_cov_2019,pred_cov_2020])

In [72]:
final_cov_pred

ticker                 AAPL      AMGN       AXP  ...        VZ       WBA       WMT
Date       ticker                                ...                              
2016-01-04 AAPL    0.000321  0.000153  0.000125  ...  0.000138  0.000162  0.000107
           AMGN    0.000153  0.000269  0.000151  ...  0.000102  0.000171  0.000069
           AXP     0.000125  0.000151  0.000160  ...  0.000094  0.000118  0.000059
           BA      0.000155  0.000156  0.000169  ...  0.000107  0.000132  0.000077
           CAT     0.000148  0.000154  0.000109  ...  0.000096  0.000130  0.000098
...                     ...       ...       ...  ...       ...       ...       ...
2020-12-30 TRV     0.000058  0.000111  0.000191  ...  0.000066  0.000119  0.000012
           UNH     0.000141  0.000220  0.000202  ...  0.000130  0.000140 -0.000002
           VZ      0.000024  0.000083  0.000108  ...  0.000185  0.000080  0.000006
           WBA     0.000003  0.000153  0.000198  ...  0.000080  0.000279  0.000050
           WMT     0.000069  0.000015  0.000079  ...  0.000006  0.000050  0.000156

[33966 rows x 27 columns]

In [73]:
final_cov_pred.to_csv("cov_forecast.csv")